In [ ]:
import torch
import tinycudann as tcnn
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import BSpline

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Optimize 2D path

In [ ]:
# Path parameterization
t = [0, 1, 2, 3, 4, 5, 6]
k = 2
c = [-1, 2, 0, -1]
xspl = BSpline(t, c, k)
spl(0.0)

NEM MLP and encoding network

In [ ]:
encoding = [tcnn.Encoding(
    n_input_dims=2,
    encoding_config={
        "otype": "HashGrid",
        "n_levels": 16,
        "n_features_per_level": 8,
        "log2_hashmap_size": 19,
        "base_resolution": 16,
        "per_level_scale": 1.2599210739135742,
    },
)]

In [ ]:
tot_out_dims_2d = 128

heightcap_net = tcnn.Network(
    n_input_dims=tot_out_dims_2d,
    n_output_dims=1,
    network_config={
        "otype": "CutlassMLP",
        "activation": "ReLU",
        "output_activation": "None",
        "n_neurons": 256,
        "n_hidden_layers": 1,
    },
)

Load trained net

In [ ]:
heightcap_net.load_state_dict(torch.load('san_jose_heightcap_net.pth'))

Generate point cloud

In [ ]:
# [-1, 1] x [-1, 1] xy grid
N = 256
XY_grid = torch.meshgrid(
    torch.linspace(-1, 1, N),
    torch.linspace(-1, 1, N),
)
XY_grid = torch.stack(XY_grid, dim=-1).to(device)
positions = XY_grid.reshape(-1, 2)

In [ ]:
xs = [e(positions) for e in encoding]
x = torch.concat(xs, dim=-1)

heights = heightcap_net(x)

In [ ]:
# From dataparser_transforms.json
#scale = 0.19145998794537697  # unreal_moon
scale = 5.6422821338546804e-06  # ges_moon

In [ ]:
xy = positions.cpu().detach().numpy()
x = xy[:,0] 
y = xy[:,1] 
z = heights.cpu().detach().numpy().flatten() 

In [ ]:
# fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=2))])
# Color by z value
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=2, color=z, colorscale='Viridis'))])
fig.update_layout(title='Elevation Model', width=1500, height=800)
fig.update_layout(scene_aspectmode='data')
fig.show()

Get elevation from lat long

In [ ]:
import requests
import urllib
import pandas as pd

# USGS Elevation Point Query Service
#url = r'https://nationalmap.gov/epqs/pqs.php?'
#new 2023:
url = r'https://epqs.nationalmap.gov/v1/json?'

# coordinates with known elevation 
lat = [48.633, 48.733, 45.1947, 45.1962]
lon = [-93.9667, -94.6167, -93.3257, -93.2755]
   
# create data frame
df = pd.DataFrame({
    'lat': lat,
    'lon': lon
})

def elevation_function(df, lat_column, lon_column):
    """Query service using lat, lon. add the elevation values as a new column."""
    elevations = []
    for lat, lon in zip(df[lat_column], df[lon_column]):
                
        # define rest query params
        params = {
            'output': 'json',
            'x': lon,
            'y': lat,
            'units': 'Meters'
        }
        
        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        #elevations.append(result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])
        #new 2023:
        elevations.append(result.json()['value'])

    df['elev_meters'] = elevations

elevation_function(df, 'lat', 'lon')
df.head()